In [1]:
import datetime
import glob
import requests
import ast
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pytickersymbols import PyTickerSymbols

# News preprocessing

In [2]:
# get the absolute path of all Excel files 
rwe_news = glob.glob('../data/lexisnexis/rwe/*.xlsx')

# read all Excel files at once
rwe_news = pd.concat([pd.read_excel(file) for file in rwe_news])
display(rwe_news)
rwe_news = rwe_news[['Date', 'Headline']]
rwe_news = rwe_news.rename(columns={
    'Date': 'time_stamp',
    'Headline': 'title'
})

rwe_news = rwe_news.dropna()
rwe_news = rwe_news.sort_values('time_stamp')
rwe_news = rwe_news.drop_duplicates(keep='first')
#searchfor = ('rwe', 'innogy')
#rwe_news = rwe_news[rwe_news['title'].str.lower().str.contains('|'.join(searchfor))]
rwe_news = rwe_news[rwe_news['title'].str.lower().str.contains('rwe')]
rwe_news = rwe_news[rwe_news['title'].str.len() > 15]
rwe_news = rwe_news.reset_index(drop=True)

display(rwe_news)

rwe_news.to_csv('../data/rwe_news_prep.csv', sep=';', index=False)

,Headline,Publication,Summary,Date
0,RWE AG - 2010,"Germany Mining Report, 772 words",NaN,2010-07-20
1,RWE AG - 2010,"Germany Mining Report, 766 words",NaN,2010-01-01
2,RWE AG - 2011,"Germany Mining Report, 822 words",NaN,2011-01-01
3,DGAP-CMS: RWE AG : Release of a capital market...,"AWP OTS (Original text service) - English, ADH...",NaN,2010-09-30
4,"RWE AG LNG Export and Import Markets, 2000 to ...","M2 PressWIRE, 471 words",NaN,2010-09-30
...,...,...,...,...
985,University of Warsaw awards electricity distri...,"SeeNews Poland, 135 words",NaN,2014-05-27
986,(1/4) Results for the Three and Nine Months En...,"Market News Publishing, 6457 words",NaN,2013-11-07
987,"Global Fuel Cells Market, Global Industry Anal...","PR Newswire, 6414 words",NaN,2014-10-16
988,"Global Fuel Cells Market, Global Industry Anal...","PR Newswire, 6422 words",NaN,2014-07-09


,time_stamp,title
0,2008-01-03,Municipal Investors Bundle Stakes in German RWE
1,2008-01-04,"German RWE Eprimo with 87,000 Customers in 2007"
2,2008-01-04,RWE AG 's npower hikes UK gas prices by 17.2 p...
3,2008-01-04,"RWE 's discount unit Eprimo adds 87,000 new cu..."
4,2008-01-09,Germany 's RWE to up stake in Budapest power c...
...,...,...
3075,2022-12-29,"Thursday December 29, 2022 RWE ADR Compendium ..."
3076,2022-12-29,- Sempra Infrastructure signs long-term sale a...
3077,2022-12-29,Sempra subsidiary finalizes 15-year deal to su...
3078,2022-12-29,RWE ADR (RWEOY: $44.40) increases 1.6% strengt...


# Stock Price preprocessing

In [3]:
def get_stock_price(symbol, start='2010-01-01', end='2022-10-31'):
    url = "https://yahoo-finance97.p.rapidapi.com/price-customdate"

    payload = f"end={end}&symbol={symbol}&start={start}"
    headers = {
        "content-type": "application/x-www-form-urlencoded",
        "X-RapidAPI-Key": "",
        "X-RapidAPI-Host": "yahoo-finance97.p.rapidapi.com"
    }
    response = requests.request("POST", url, data=payload, headers=headers)
    
    df = response.text
    df = ast.literal_eval(df)
    df = df['data']
    df = pd.DataFrame(df)
    #display(df)
    df['Date'] = df['Date'].apply(lambda x: datetime.datetime.fromtimestamp(x/1000).date())
    df.columns = [x.lower() for x in df.columns]
    df = df[[
        'date',
        'close'
    ]]
    
    return df

#rwe_price = get_stock_price('^GDAXI', start='2008-01-01', end='2022-12-31')
#rwe_price.to_csv('../data/dax_prices_raw.csv', sep=';', index=False)

In [4]:
def stock_ts_prep(path):
    df = pd.read_csv(path, sep=';', index_col='date')
    df.index.name = 'time_stamp'
    
    df = df.rename(columns={
        'close': 'price'
    })

    df.index = pd.to_datetime(df.index)

    df = df[df.index.dayofweek < 5]

    df = df.fillna(
        method='ffill',
        limit=5
    ).fillna(
        method='bfill',
        limit=5
    )

    df = np.log(df) - np.log(df.shift(1))
    df = df.fillna(0)

    return df

rwe_price = stock_ts_prep('../data/rwe_prices_raw.csv')
rwe_price.to_csv('../data/rwe_prices_prep.csv', sep=';')
display(rwe_price)

dax_price = stock_ts_prep('../data/dax_prices_raw.csv')
dax_price.to_csv('../data/dax_prices_prep.csv', sep=';')
display(dax_price)

rwe_news = pd.read_csv('../data/rwe_news_prep.csv', sep=';')

,price
time_stamp,
2008-01-02,0.000000
2008-01-03,0.027555
2008-01-04,-0.004095
2008-01-07,0.031409
2008-01-08,-0.011701
...,...
2022-12-23,-0.005516
2022-12-27,-0.000722
2022-12-28,-0.006035


,price
time_stamp,
2008-01-02,0.000000
2008-01-03,-0.005133
2008-01-04,-0.012690
2008-01-07,0.001085
2008-01-08,0.004190
...,...
2022-12-23,0.001929
2022-12-27,0.003878
2022-12-28,-0.004978
